In [1]:
import os
import argparse
import numpy as np
import pandas as pd
import time

import torch
from torch import optim
import torch.nn as nn

from torch.utils.data import DataLoader
import torch.nn.functional as F

import torchvision.datasets as datasets
from torchvision import transforms
from data_loader import *
from model import *
from query import *

In [2]:
DEFAULT_DATASET_DIR = "./dataset/"
DEFAULT_BATCH_SIZE = 64
DEFAULT_NO_EPOCHS = 50
DEFAULT_MODEL_PATH = "./ckpt/exp_4/model_ckpt.tar"
config = Config(DEFAULT_DATASET_DIR, DEFAULT_BATCH_SIZE, DEFAULT_NO_EPOCHS)

In [3]:
images_folder_dataset = datasets.ImageFolder(root=config.images_dir+'signatures/valid')
siamese_dataset = SignatureDataset(dataset=images_folder_dataset)
print(len(siamese_dataset))

144


In [31]:
img0_file = siamese_dataset.genuine[0][1]
img1_file = siamese_dataset.genuine[1][1]
img0 = Image.open(img0_file).convert("L")
img1 = Image.open(img1_file).convert("L")
 
img0_file, img1_file

('./dataset/signatures/valid\\full_org\\original_50_1.png',
 './dataset/signatures/valid\\full_org\\original_50_10.png')

In [32]:
genuine_sigs = siamese_dataset.genuine
forged_sigs = siamese_dataset.forged

In [33]:
len(siamese_dataset)

144

In [34]:
query_model = QueryModel(DEFAULT_MODEL_PATH)

D:\Codes\axis-ai-challenge\model\model.py:72: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight, mode='fan_in')


In [35]:
output = query_model.predict(img0, img1)

In [37]:
print(output.item())

0.6463775634765625


In [47]:
img0_file = siamese_dataset.genuine[63][1]
img1_file = siamese_dataset.forged[63][1]
img0 = Image.open(img0_file).convert("L")
img1 = Image.open(img1_file).convert("L")
 
img0_file, img1_file

('./dataset/signatures/valid\\full_org\\original_52_23.png',
 './dataset/signatures/valid\\full_forg\\forgeries_52_23.png')

In [48]:
output = query_model.predict(img0, img1)

In [49]:
print(output.item())

0.03690771013498306


In [43]:
img0_file = 'vin_real.jpg'
img1_file = 'vin_forged.jpg'
img0 = Image.open(img0_file).convert("L")
img1 = Image.open(img1_file).convert("L")
 
img0_file, img1_file

('vin_real.jpg', 'vin_forged.jpg')

In [14]:
output = query_model.predict(img0, img1)

In [15]:
print(output.item())

0.9850560426712036


# CEDAR Dataset Scores

In [50]:
images_folder_dataset = datasets.ImageFolder(root=config.images_dir+'signatures/valid')
siamese_dataset = SignatureDataset(dataset=images_folder_dataset)

In [51]:
len(siamese_dataset)

144

In [52]:
total = len(siamese_dataset)
tp = 0
tn = 0
fp = 0
fn = 0
# for i in range(1, 10):
for x1, x2, label in siamese_dataset:
    print(label)
    print()
    x1 = x1.unsqueeze_(0).cuda()
    x2 = x2.unsqueeze_(0).cuda()
    x1, x2 = Variable(x1, volatile=True), Variable(x2, volatile=True)

    batch_size = x1.shape[0]

    # compute log probabilities
    output = query_model.getConfidence(x1, x2)
    print(output.item())
#     log_probas = F.sigmoid(output)
#     print(output.item())
#     log_probas = F.sigmoid(output)
#     print(log_probas.item())

    if output >= 0.5 and label == 1:
        tp += 1
        print("TRUE POSITIVE")
    elif output <= 0.5 and label == 0:
        tn += 1
        print("TRUE NEGATIVE")
    elif output >= 0.5 and label == 0:
        fp += 1
        print("FALSE POSITIVE")
    elif output <= 0.5 and label == 1:
        fn += 1
        print("FALSE NEGATIVE")

    print()

tensor([ 1.])



C:\Users\arush\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if sys.path[0] == '':


0.780139684677124
TRUE POSITIVE

tensor([ 0.])

0.6199601888656616
FALSE POSITIVE

tensor([ 1.])

0.5121605396270752
TRUE POSITIVE

tensor([ 1.])

0.6760797500610352
TRUE POSITIVE

tensor([ 0.])

0.31978392601013184
TRUE NEGATIVE

tensor([ 0.])

0.04666416719555855
TRUE NEGATIVE

tensor([ 1.])

0.30988016724586487
FALSE NEGATIVE

tensor([ 1.])

0.6864722967147827
TRUE POSITIVE

tensor([ 1.])

0.3214269280433655
FALSE NEGATIVE

tensor([ 1.])

0.19504082202911377
FALSE NEGATIVE

tensor([ 0.])

0.5971798896789551
FALSE POSITIVE

tensor([ 0.])

0.17576688528060913
TRUE NEGATIVE

tensor([ 0.])

0.09305007755756378
TRUE NEGATIVE

tensor([ 1.])

0.8329185247421265
TRUE POSITIVE

tensor([ 0.])

0.0583714134991169
TRUE NEGATIVE

tensor([ 1.])

0.43952909111976624
FALSE NEGATIVE

tensor([ 1.])

0.37679311633110046
FALSE NEGATIVE

tensor([ 1.])

0.5919651389122009
TRUE POSITIVE

tensor([ 0.])

0.13686087727546692
TRUE NEGATIVE

tensor([ 0.])

0.39421603083610535
TRUE NEGATIVE

tensor([ 0.])

0.14

In [53]:
# tp = tp/10
# fp = fp/10
# tn = tn/10
# fn = fn/10
print(tp, fp, tn, fn, total)
correct = tp+tn
print(100*correct/total)

print((tn/total)*100, (fp/total)*100)
print((fn/total)*100, (tp/total)*100)


63 4 58 19 144
84.02777777777777
40.27777777777778 2.7777777777777777
13.194444444444445 43.75


# Sample Dataset from Axis AI Scores

In [59]:
images_folder_dataset = datasets.ImageFolder(root=config.images_dir+'dataset3/train')
siamese_dataset = SignatureDataset(dataset=images_folder_dataset, dataset_type='sample')

In [60]:
len(siamese_dataset)

140

In [61]:
total = len(siamese_dataset)
tp = 0
tn = 0
fp = 0
fn = 0
# for i in range(0, 10):
for x1, x2, label in siamese_dataset:
    print(label)
    print()
    x1 = x1.unsqueeze_(0).cuda()
    x2 = x2.unsqueeze_(0).cuda()
    x1, x2 = Variable(x1, volatile=True), Variable(x2, volatile=True)

    batch_size = x1.shape[0]

    # compute log probabilities
    output = query_model.model(x1, x2)
    print(output.item())
    log_probas = F.sigmoid(output)
    print(log_probas.item())

    if log_probas >= 0.5 and label == 1:
        tp += 1
        print("TRUE POSITIVE")
    elif log_probas <= 0.5 and label == 0:
        tn += 1
        print("TRUE NEGATIVE")
    elif log_probas >= 0.5 and label == 0:
        fp += 1
        print("FALSE POSITIVE")
    elif log_probas <= 0.5 and label == 1:
        fn += 1
        print("FALSE NEGATIVE")

    print()


tensor([ 0.])



C:\Users\arush\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if sys.path[0] == '':


-5.370221138000488
0.00463155098259449
TRUE NEGATIVE

tensor([ 1.])

0.04865184426307678
0.5121605396270752
TRUE POSITIVE

tensor([ 0.])

-1.6223682165145874
0.16487853229045868
TRUE NEGATIVE

tensor([ 0.])

-4.152367115020752
0.015483630821108818
TRUE NEGATIVE

tensor([ 0.])

-5.760610103607178
0.003139303531497717
TRUE NEGATIVE

tensor([ 1.])

1.4919047355651855
0.8163639903068542
TRUE POSITIVE

tensor([ 0.])

-4.21159029006958
0.01460627093911171
TRUE NEGATIVE

tensor([ 0.])

0.42453858256340027
0.6045687794685364
FALSE POSITIVE

tensor([ 0.])

-3.4011285305023193
0.032260213047266006
TRUE NEGATIVE

tensor([ 1.])

0.6727436780929565
0.6621172428131104
TRUE POSITIVE

tensor([ 1.])

2.477046489715576
0.922516942024231
TRUE POSITIVE

tensor([ 0.])

-2.8387224674224854
0.055267203599214554
TRUE NEGATIVE

tensor([ 0.])

0.2800031006336212
0.5695469975471497
FALSE POSITIVE

tensor([ 0.])

7.347695827484131
0.9993563294410706
FALSE POSITIVE

tensor([ 1.])

4.133667469024658
0.9842287302017

1.072258710861206
0.7450262308120728
FALSE POSITIVE

tensor([ 0.])

0.9122428894042969
0.7134588956832886
FALSE POSITIVE

tensor([ 0.])

-3.219390630722046
0.03844250366091728
TRUE NEGATIVE

tensor([ 0.])

-1.8152321577072144
0.14000695943832397
TRUE NEGATIVE

tensor([ 0.])

1.134006142616272
0.7565774917602539
FALSE POSITIVE

tensor([ 1.])

0.04865184426307678
0.5121605396270752
TRUE POSITIVE

tensor([ 1.])

1.5637092590332031
0.826884925365448
TRUE POSITIVE

tensor([ 1.])

-2.1216349601745605
0.10701172798871994
FALSE NEGATIVE

tensor([ 0.])

-10.043386459350586
4.347041613073088e-05
TRUE NEGATIVE

tensor([ 1.])

-2.440416097640991
0.08014222979545593
FALSE NEGATIVE

tensor([ 1.])

0.04865184426307678
0.5121605396270752
TRUE POSITIVE

tensor([ 0.])

3.390523672103882
0.9674070477485657
FALSE POSITIVE

tensor([ 0.])

3.106064558029175
0.9571422338485718
FALSE POSITIVE

tensor([ 1.])

0.04865184426307678
0.5121605396270752
TRUE POSITIVE

tensor([ 0.])

5.060824871063232
0.9936995506286

In [62]:
# tp = tp/10
# fp = fp/10
# tn = tn/10
# fn = fn/10
correct = tp+tn
print(100*correct/total)

print(((tp)/total)*100, ((fp)/total)*100)
print(((tn)/total)*100, ((fn)/total)*100)


70.71428571428571
43.57142857142857 25.0
27.142857142857142 4.285714285714286


# BHSig Scores


In [58]:
images_folder_dataset = datasets.ImageFolder(root=config.images_dir+'BHSig260/')
siamese_dataset = SignatureDataset(dataset=images_folder_dataset, dataset_type='bhsig')

KeyboardInterrupt: 

In [ ]:
total = len(siamese_dataset)
tp = 0
tn = 0
fp = 0
fn = 0

for i, (x1, x2, label) in enumerate(siamese_dataset):
    start = time.time()
    print(label)
    print(i)
    x1 = x1.unsqueeze_(0).cuda()
    x2 = x2.unsqueeze_(0).cuda()
    x1, x2 = Variable(x1, volatile=True), Variable(x2, volatile=True)

    batch_size = x1.shape[0]

    # compute log probabilities
    output = query_model.model(x1, x2)
    log_probas = F.sigmoid(output)
    print(output.item())
    log_probas = F.sigmoid(output)
    print(log_probas.item())

    if log_probas >= 0.5 and label == 1:
        tp += 1
        print("TRUE POSITIVE")
    elif log_probas <= 0.5 and label == 0:
        tn += 1
        print("TRUE NEGATIVE")
    elif log_probas >= 0.5 and label == 0:
        fp += 1
        print("FALSE POSITIVE")
    elif log_probas <= 0.5 and label == 1:
        fn += 1
        print("FALSE NEGATIVE")
        
    print(time.time() - start)
    print()

In [ ]:
print(total)
print(tp, tn)
print(fp, fn)
print(tp+tn+fp+fn)
correct = tp+tn
print(100*correct/(tp+tn+fp+fn))

print(((tp/1)/total)*100, ((fp/1)/total)*100)
print(((tn/1)/total)*100, ((fn/1)/total)*100)
